In [ ]:
# Resources
# U-NET: https://arxiv.org/pdf/1505.04597.pdf 
# Semantic segmentation: https://youtu.be/csFGTLT6_WQ
# Annotation tool: https://app.cvat.ai/tasks?page=1 

In [ ]:
# Create masks of SEM images using the annotation tool 
# Iterate through and change masks to binary

from scipy import ndimage, misc
import numpy as np
import os
import cv2
import imageio.v2 as imageio

def main():
    outPath = "C:/Users/"
    path = "C:/Users/"

    # iterate through the mask images of the folder
    for image_path in os.listdir(path):

        # create the full input path and read the image
        input_path = os.path.join(path, image_path)
        image_to_mask = imageio.imread(input_path)

        # change masks to binary 
        binary = np.where(image_to_mask>0, 255, image_to_mask)

        # create full output path, 'example.jpg' 
        # becomes 'rotate_example.jpg', save the file to disk
        fullpath = os.path.join(outPath, 'binary_'+image_path)
        imageio.imwrite(fullpath, binary)

if __name__ == '__main__':
    main()

In [ ]:
# Iterate through and resize all SEM images and binary masks to fit U-NET model

from simple_unet_model import simple_unet_model   
from keras.utils import normalize
import os
import cv2
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

image_directory = 'C:/Users/'
mask_directory = 'C:/Users/'

SIZE = 256
image_dataset = []  
mask_dataset = []  

images = os.listdir(image_directory)
for i, image_name in enumerate(images):    
    if (image_name.split('.')[1] == 'tiff'):
        image = cv2.imread(image_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        image_dataset.append(np.array(image))
        
masks = os.listdir(mask_directory)
for i, image_name in enumerate(masks):
    if (image_name.split('.')[1] == 'png'):
        image = cv2.imread(mask_directory+image_name, 0)
        image = Image.fromarray(image)
        image = image.resize((SIZE, SIZE))
        mask_dataset.append(np.array(image))

In [ ]:
# Normalize all SEM images and binary masks

image_dataset = np.expand_dims(normalize(np.array(image_dataset), axis=1),3)
mask_dataset = np.expand_dims((np.array(mask_dataset)),3) /255.

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(image_dataset, mask_dataset, test_size = 0.10, random_state = 0)

In [ ]:
# run the U-NET model

IMG_HEIGHT = image_dataset.shape[1]
IMG_WIDTH  = image_dataset.shape[2]
IMG_CHANNELS = image_dataset.shape[3]

def get_model():
    return simple_unet_model(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)

model = get_model()

history = model.fit(X_train, y_train, 
                    batch_size = 16, 
                    verbose=1, 
                    epochs=200, 
                    validation_data=(X_test, y_test), 
                    shuffle=False)

model.save('file.hdf5')

In [ ]:
# IOU score

y_pred=model.predict(X_test)
y_pred_thresholded = y_pred > 0.5

intersection = np.logical_and(y_test, y_pred_thresholded)
union = np.logical_or(y_test, y_pred_thresholded)
iou_score = np.sum(intersection) / np.sum(union)
print("IoU score is: ", iou_score)